In [1]:
%load_ext micropython_magic
%reload_ext micropython_magic

Current ip.InteractiveTB.mode='Context'
New ip.InteractiveTB.mode='Minimal'
Current ip.InteractiveTB.mode='Minimal'


In [13]:
# %mpy -s {"/dev/cu.usbmodem_fs3_1"}
%mpy -s {"/dev/cu.usbmodem1101"}

In [19]:
# %%micropython

import time
from config import config
from machine import RTC
from mp_libs.network import Network
from mp_libs.protocols import espnow_protocol
from mp_libs.time import ptp

NUM_SYNC_CYCLES = 15
TIMEOUT_MSEC = 5000

rtc = RTC()
miniot_network = Network.create_min_iot("GTY")
epn = miniot_network.transport.transport._transport
miniot_network.connect()

print(f"Initial RTC datetime: {rtc.datetime()}")
print(f"Initial RTC now: {rtc.now()}")
print(f"My MAC: {epn.wifi._sta.config('mac')}")
print(f"EPN Peer: {config['epn_peer_mac']}")
print(f"Initial channel: {epn.wifi._sta.config('channel')}")

# Perform espnow scan to set the right channel
miniot_network.scan()
print(f"Final channel: {epn.wifi._sta.config('channel')}")
print(f"EPN timeout: {epn.epn.config('timeout_ms')}")

while True:
    # Perform periph sequence
    timestamps = ptp.sequence_periph(
        miniot_network,
        lambda miniot_msg: miniot_msg.msg,
        TIMEOUT_MSEC,
        initiate_sync=True,
        num_sync_cycles=NUM_SYNC_CYCLES)

    # Calculate offsets
    offsets = [ptp.calculate_offset(t1, t2, t3, t4) for t1, t2, t3, t4 in timestamps]

    print(f"timestamps: {timestamps}")
    print(f"offsets: {offsets}")

    # Process offsets
    ave_offset = ptp.process_offsets(offsets)

    # Update time
    # rtc.offset(str(ave_offset).encode())
    rtc.offset(ave_offset)

    print(f"ave_offset: {ave_offset}")
    print(f"ave_offset bytes: {str(ave_offset).encode()}")
    print(f"now: {rtc.datetime()}")
    print(f"RTC now: {rtc.now()}")

    time.sleep(15)


2000-01-01 00:00:04.4253 INFO-wifi-protocols:Disconnecting wifi...
2000-01-01 00:00:04.4257 INFO-miniot:Connecting MinIoT...
2000-01-01 00:00:04.4259 INFO-espnow-protocol:Connecting espnow...
Initial RTC datetime: (2000, 1, 1, 5, 0, 0, 4, 261786)
Initial RTC now: 4262534
My MAC: b'p\x04\x1d\xad|\xc0'
EPN Peer: b"H'\xe2\xc7n\xfc"
Initial channel: 1
2000-01-01 00:00:04.4267 INFO-wifi-protocols:Disconnecting wifi...
2000-01-01 00:00:04.4817 INFO-wifi-protocols:Disconnecting wifi...
2000-01-01 00:00:05.5348 INFO-wifi-protocols:Disconnecting wifi...
2000-01-01 00:00:05.5877 INFO-wifi-protocols:Disconnecting wifi...
2000-01-01 00:00:06.6407 INFO-wifi-protocols:Disconnecting wifi...
2000-01-01 00:00:06.6937 INFO-wifi-protocols:Disconnecting wifi...
2000-01-01 00:00:07.7478 INFO-wifi-protocols:Disconnecting wifi...
2000-01-01 00:00:08.8007 INFO-wifi-protocols:Disconnecting wifi...
2000-01-01 00:00:08.8547 INFO-wifi-protocols:Disconnecting wifi...
Final channel: 9
EPN timeout: 0
timestamps: [(7